In [14]:
import requests
import json
import time
from requests.adapters import HTTPAdapter, Retry

# Setup session with retry handling
session = requests.Session()
retry = Retry(total=5, backoff_factor=1, status_forcelist=[429, 500, 502, 503, 504])
adapter = HTTPAdapter(max_retries=retry)
session.mount("https://", adapter)

# Fetch all results for a given lat/lon
def fetch_all_foursquare_restaurants(lat, lon, api_key, radius=5000, limit=50):
    all_results = []
    url = "https://api.foursquare.com/v3/places/search"
    headers = {
        "Accept": "application/json",
        "Authorization": api_key,
        "User-Agent": "Mozilla/5.0"
    }

    params = {
        "ll": f"{lat},{lon}",
        "radius": radius,
        "limit": limit,
        "categories": "13065"  # Category ID for restaurants
    }

    next_cursor = None
    previous_cursor = None
    attempts = 0

    while True:
        if next_cursor:
            params["cursor"] = next_cursor

        try:
            response = session.get(url, headers=headers, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()

            results = data.get("results", [])
            all_results.extend(results)
            print(f"Fetched {len(results)} results (Total so far: {len(all_results)})")

            next_cursor = data.get("context", {}).get("next_cursor")

            # Exit conditions
            if not results or next_cursor == previous_cursor:
                print("🔚 No more results or repeated cursor. Breaking loop.")
                break

            previous_cursor = next_cursor
            attempts += 1
            if attempts > 10:
                print("🚫 Max pagination reached. Breaking.")
                break

            time.sleep(1)

        except requests.exceptions.RequestException as e:
            print(f"❌ Request failed: {e}")
            break

    return all_results

# Replace with your actual Foursquare API key
api_key = "fsq3RZiQAOeHzAEslLwurjhnkL23N8jH+CwkZILirp836wg="  # e.g., "fsq3..."

# Coordinates for multiple subareas per city
city_points = {
    "New York": [
        (40.7128, -74.0060), (40.7306, -73.9352),
        (40.7851, -73.9683), (40.7488, -73.9857)
    ],
    "San Francisco": [
        (37.7749, -122.4194), (37.7599, -122.4148),
        (37.7986, -122.4075), (37.7841, -122.4089)
    ],
    "Chicago": [
        (41.8781, -87.6298), (41.7950, -87.5800),
        (41.9000, -87.6500), (41.8500, -87.7000)
    ],
    "Boston": [
        (42.3601, -71.0589), (42.3467, -71.0972),
        (42.3736, -71.1097), (42.3320, -71.0876)
    ],
    "Los Angeles": [
        (34.0522, -118.2437), (34.0736, -118.4004),
        (34.0194, -118.4912), (34.0407, -118.2468)
    ]
}

# Main loop to fetch and save data
total_records = 0

for city, coords_list in city_points.items():
    city_all_data = []
    print(f"\n🔍 Fetching Foursquare restaurants in {city}...")

    for lat, lon in coords_list:
        subarea_data = fetch_all_foursquare_restaurants(lat, lon, api_key)
        city_all_data.extend(subarea_data)
        time.sleep(1)

    filename = f"{city.replace(' ', '_').lower()}_restaurants_foursquare.json"
    with open(filename, 'w') as f:
        json.dump(city_all_data, f, indent=2)

    total_records += len(city_all_data)

print(f"\n✅ Done! Total restaurants collected: {total_records}")



🔍 Fetching Foursquare restaurants in New York...
Fetched 50 results (Total so far: 50)
🔚 No more results or repeated cursor. Breaking loop.
Fetched 50 results (Total so far: 50)
🔚 No more results or repeated cursor. Breaking loop.
Fetched 50 results (Total so far: 50)
🔚 No more results or repeated cursor. Breaking loop.
Fetched 50 results (Total so far: 50)
🔚 No more results or repeated cursor. Breaking loop.

🔍 Fetching Foursquare restaurants in San Francisco...
Fetched 50 results (Total so far: 50)
🔚 No more results or repeated cursor. Breaking loop.
Fetched 50 results (Total so far: 50)
🔚 No more results or repeated cursor. Breaking loop.
Fetched 50 results (Total so far: 50)
🔚 No more results or repeated cursor. Breaking loop.
Fetched 50 results (Total so far: 50)
🔚 No more results or repeated cursor. Breaking loop.

🔍 Fetching Foursquare restaurants in Chicago...
Fetched 50 results (Total so far: 50)
🔚 No more results or repeated cursor. Breaking loop.
Fetched 50 results (Total s